In [4]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timezone

KEY = 'eyJvcmciOiI2NDA2NTFhNTIyZmEwNTAwMDEyOWJiZTEiLCJpZCI6IjlkMGQ5NzA1ZDNjMDRhNWU4YTVhZjJkMWY2OTgxYzkwIiwiaCI6Im11cm11cjEyOCJ9'
ORIGIN = (46.975966, 7.456462)
DESTINATION = (47.380656, 8.541437)
VIA = (47.384094, 8.240575)
ARR = datetime(2025, 3, 27, 16, 30, 0).isoformat()
MODE = 'car'
TIMESTAMP = datetime.now(timezone.utc).astimezone().isoformat()

def create_ojp_trip_request(timestamp=TIMESTAMP, origin=ORIGIN, destination=DESTINATION, via=VIA, arr=ARR, mode=MODE, num_results=3, include_stops=True, include_track_sect=False, include_leg_proj=False, include_turn_desc=False):
    """
    Constructs an OJP TripRequest XML.
    """
    # Read the XML template from file
    with open("templates/ojp_request_template.xml", "r", encoding="utf-8") as file:
        xml_template = file.read()

    # Format XML correctly
    xml_request = xml_template.format(
        timestamp=timestamp,
        origin=origin,
        destination=destination,
        via=via,
        arr=arr,
        num_results=num_results,
        include_track_sect=str(include_track_sect).lower(),
        include_stops=str(include_stops).lower(),
        include_leg_proj=str(include_leg_proj).lower(),
        include_turn_desc=str(include_turn_desc).lower()
    )

    # Save request to a file for debugging purposes
    with open("templates/ojp_trip_request.xml", "w", encoding="utf-8") as file:
        file.write(xml_request)
    return xml_request

def send_ojp_request(xml_request, endpoint):
    """
    Sends the OJP TripRequest to the server.
    """
    headers = {'Content-Type': 'application/xml', 
               'Authorization': f'Bearer {KEY}'}
    response = requests.post(endpoint, data=xml_request, headers=headers)
    return response.text

endpoint = "https://api.opentransportdata.swiss/ojp20"
xml_request = create_ojp_trip_request()
response_xml = send_ojp_request(xml_request, endpoint)
print(response_xml[:1000])

<html>
    <head>
        <title>ODMCH OJP Service Unavailable</title>
    </head>
    <body>
        <h1>ODMCH OJP Service Unavailable</h1>
Our apologies for the temporary inconvenience. We are making sure the service is up and running as fast as possible. 
    </body>
</html>


In [ ]:
def parse_ojp_response(response_xml):
    """
    Parses the OJP TripResponse XML to extract journey details.
    """
    root = ET.fromstring(response_xml)
    namespaces = {'ojp': 'http://www.vdv.de/trias'}
    journeys = []
    
    for journey in root.findall(".//ojp:TripResult", namespaces):
        trip_info = {}
        
        origin = journey.find(".//ojp:Origin/ojp:LocationName/ojp:Text", namespaces)
        destination = journey.find(".//ojp:Destination/ojp:LocationName/ojp:Text", namespaces)
        duration = journey.find(".//ojp:Duration", namespaces)
        
        trip_info['origin'] = origin.text if origin is not None else "Unknown"
        trip_info['destination'] = destination.text if destination is not None else "Unknown"
        trip_info['duration'] = duration.text if duration is not None else "Unknown"
        
        trip_info['legs'] = []
        for leg in journey.findall(".//ojp:TripLeg", namespaces):
            leg_info = {}
            mode = leg.find(".//ojp:Service/ojp:Mode/ojp:PtMode", namespaces)
            dep_time = leg.find(".//ojp:LegBoard/ojp:ServiceDeparture/ojp:TimetabledTime", namespaces)
            arr_time = leg.find(".//ojp:LegAlight/ojp:ServiceArrival/ojp:TimetabledTime", namespaces)
            
            leg_info['mode'] = mode.text if mode is not None else "Unknown"
            leg_info['departure_time'] = dep_time.text if dep_time is not None else "Unknown"
            leg_info['arrival_time'] = arr_time.text if arr_time is not None else "Unknown"
            
            trip_info['legs'].append(leg_info)
        
        journeys.append(trip_info)
    
    return journeys

journeys = parse_ojp_response(response_xml)
print(journeys)